<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/criar_base_local_passagens_com_julgamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação

In [ ]:
# Install the latest release of Haystack in your own environment 
! pip install farm-haystack

# Alternative installation the latest master of Haystack
#!pip install grpcio-tools==1.34.1
#!pip install git+https://github.com/deepset-ai/haystack.git

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

In [1]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


## Instalando e inicializando ElasticSearc

In [2]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [3]:
!curl -X GET "localhost:9200"

{
  "name" : "07239d2ca8a9",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "mMgt9yQ_TpCcQwLpvZOF4A",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [4]:
!curl -X GET "localhost:9200"

{
  "name" : "b2fe8e2b2fe7",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "M4nvSvHMRYaMAyeD0ccfQg",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [4]:
doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='robustez-query'
)

11/09/2021 16:55:34 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.012s]
11/09/2021 16:55:34 - INFO - elasticsearch -   PUT http://localhost:9200/robustez-query [status:200 request:0.334s]
11/09/2021 16:55:35 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.195s]


In [5]:
import requests

In [6]:
requests.get('http://localhost:9200/_cluster/health').json()

{'active_primary_shards': 2,
 'active_shards': 2,
 'active_shards_percent_as_number': 50.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'yellow',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 2}

In [8]:
requests.get('http://localhost:9200/_cat/indices')

<Response [200]>

## Carga do dataset MsMarco Passage

In [ ]:
#from pyserini.search import SimpleSearcher

# `SimpleSearcher` defaults to BM25 scoring function.
#searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

## Carga das queries originais

In [9]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/passages-with-judment.txt

--2021-11-09 17:01:18--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/passages-with-judment.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3844638 (3.7M) [text/plain]
Saving to: ‘passages-with-judment.txt’

passages-with-judme 100%[===================>]   3.67M  --.-KB/s    in 0.07s   

2021-11-09 17:01:19 (53.1 MB/s) - ‘passages-with-judment.txt’ saved [3844638/3844638]



In [10]:
passages = []

for passage in open('passages-with-judment.txt'):
    fields = passage.strip().split()
    passages.append((fields[0], ' '.join(fields[1:])))

In [12]:
passages[0][0], passages[0][1]

('1020327',
 'From U.S. Citizenship and Immigration Services (USCIS) Naturalization Guide to Naturalization Child Citizenship Act Naturalization Test. Laws and Regulations Read the Code of Federal Regulation Chapter 8 Section 319.2, Expeditious Naturalization regulation and read the INA section 319(b). Department of State Employees and Spouses Only')

In [18]:
data_json = [
    {
        'text': passage[1],
        'meta': {
            'docid': passage[0],
            'source': 'msmarco-passage-with-judment'
        }
    } for passage in passages
]

In [19]:
data_json[0]

{'meta': {'docid': '1020327', 'source': 'msmarco-passage-with-judment'},
 'text': 'From U.S. Citizenship and Immigration Services (USCIS) Naturalization Guide to Naturalization Child Citizenship Act Naturalization Test. Laws and Regulations Read the Code of Federal Regulation Chapter 8 Section 319.2, Expeditious Naturalization regulation and read the INA section 319(b). Department of State Employees and Spouses Only'}

In [20]:
len(data_json)

11224

In [21]:
doc_store.write_documents(data_json)

11/09/2021 17:07:59 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.314s]
11/09/2021 17:08:01 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.062s]
11/09/2021 17:08:02 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.014s]
11/09/2021 17:08:03 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.017s]
11/09/2021 17:08:04 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
11/09/2021 17:08:05 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.012s]
11/09/2021 17:08:06 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.012s]
11/09/2021 17:08:07 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.019s]


In [22]:
requests.get('http://localhost:9200/robustez-query/_count').json()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 11224}

## Execução de busca densa


In [23]:
from haystack.retriever.dense import DensePassageRetriever

In [24]:
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

11/09/2021 17:10:08 - DEBUG - filelock -   Attempting to acquire lock 139807006850960 on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
11/09/2021 17:10:08 - DEBUG - filelock -   Lock 139807006850960 acquired on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

11/09/2021 17:10:08 - DEBUG - filelock -   Attempting to release lock 139807006850960 on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
11/09/2021 17:10:08 - DEBUG - filelock -   Lock 139807006850960 released on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
11/09/2021 17:10:08 - DEBUG - filelock -   Attempting to acquire lock 139807002201296 on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
11/09/2021 17:10:08 - DEBUG - filelock -   Lock 139807002201296 acquired on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f35

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

11/09/2021 17:10:09 - DEBUG - filelock -   Attempting to release lock 139807002201296 on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
11/09/2021 17:10:09 - DEBUG - filelock -   Lock 139807002201296 released on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
11/09/2021 17:10:09 - DEBUG - filelock -   Attempting to acquire lock 139807007628496 on /root/.cache/huggingface/transformers/d5b5f07ee846d5baa7142e121b6ee77d11ac68bd5d4541faab38a1ea76c2954a.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
11/09/2021 17:10:09 - DEBUG - filelock -   Lock 139807007628496 acquired on /root/.cache/huggingface/transformers/d5b5f07ee846d5baa7142e121b6ee77d11ac68bd5d4541faab38a1ea76c2954a.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146

...

11/09/2021 17:10:25 - DEBUG - filelock -   Attempting to release lock 139806982218512 on /root/.cache/huggingface/transformers/70b0d7ed89bb3511a323f99b7cfa4a3e0c35754fda6a3ac74c3458ca8ffb5764.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
11/09/2021 17:10:25 - DEBUG - filelock -   Lock 139806982218512 released on /root/.cache/huggingface/transformers/70b0d7ed89bb3511a323f99b7cfa4a3e0c35754fda6a3ac74c3458ca8ffb5764.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
11/09/2021 17:10:25 - DEBUG - filelock -   Attempting to acquire lock 139806943115920 on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285d3b624cd75dd9ee6.lock
11/09/2021 17:10:25 - DEBUG - filelock -   Lock 139806943115920 acquired on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

11/09/2021 17:10:25 - DEBUG - filelock -   Attempting to release lock 139806943115920 on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285d3b624cd75dd9ee6.lock
11/09/2021 17:10:25 - DEBUG - filelock -   Lock 139806943115920 released on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285d3b624cd75dd9ee6.lock
11/09/2021 17:10:25 - DEBUG - filelock -   Attempting to acquire lock 139806987874064 on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock
11/09/2021 17:10:25 - DEBUG - filelock -   Lock 139806987874064 acquired on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d8

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

11/09/2021 17:10:39 - DEBUG - filelock -   Attempting to release lock 139806987874064 on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock
11/09/2021 17:10:39 - DEBUG - filelock -   Lock 139806987874064 released on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock


In [25]:
doc_store.update_embeddings(retriever=retriever)

11/09/2021 17:11:02 - INFO - elasticsearch -   POST http://localhost:9200/robustez-query/_count [status:200 request:0.016s]
11/09/2021 17:11:02 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for all 11224 docs ...


Updating embeddings:   0%|          | 0/11224 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1232 [00:00<?, ? Docs/s]

In [26]:
retriever.retrieve('Citizenship and Immigration Services')

[{'text': "FOR CONFIDENTIALITY OF CENSUS INFORMATION. Section 642(a) of the Illegal Immigration Reform and Immigrant Responsibility Act of. 1996, which concerns the authority of federal, state, and local government officials and entities. to disclose to the Immigration and Naturalization Service information regarding an individual's.", 'score': 0.6769241275058455, 'question': None, 'meta': {'docid': '3878669', 'source': 'msmarco-passage-with-judment'}, 'embedding': None, 'id': 'baf8de57193d6cd7724ada81850d2fb9'},
 {'text': "Naturalization is the process by which an alien becomes an American citizen. These records can provide a researcher with information such as a person's birth date and location, occupation, immigration year, marital status and spouse information, witnesses' names and addresses, and more.", 'score': 0.6762501713946443, 'question': None, 'meta': {'docid': '4556933', 'source': 'msmarco-passage-with-judment'}, 'embedding': None, 'id': 'ed450b145ec4c3468d8209d000d2ec40'},

In [ ]:
! wget -nc https://trec.nist.gov/data/deep/2020qrels-pass.txt

--2021-11-09 14:21:36--  https://trec.nist.gov/data/deep/2020qrels-pass.txt
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.175, 2610:20:6b01:4::175
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218617 (213K) [text/plain]
Saving to: ‘2020qrels-pass.txt’

2020qrels-pass.txt  100%[===================>] 213.49K   374KB/s    in 0.6s    

2021-11-09 14:21:38 (374 KB/s) - ‘2020qrels-pass.txt’ saved [218617/218617]



In [ ]:
judment = {}
for i, line in enumerate(open('2020qrels-pass.txt')):
    query_nr, _, pid, eval = line.rstrip().split()
    judment[(query_nr, pid)] = int(eval)

In [ ]:
scale = {3:'perfectly relevant', 2:'highly relevant', 1:'related', 0:'Irrelevant'}

In [ ]:
list(judment)[0], judment[list(judment)[0]]

(('23849', '1020327'), 2)

In [ ]:
!nvidia-smi

Tue Nov  9 14:21:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------